# Part 3: XGBoost Training (Top 10)
Nonlinear feature-based classification using XGBoost for 10 cryptocurrencies.

### CELL 1: Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q pandas numpy scikit-learn xgboost joblib

### CELL 2: Load & Prepare Data

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
import joblib

# Load master dataset
df = pd.read_csv('/content/drive/MyDrive/crypto_bot/data/master_top10_2y.csv')

print(f"📊 Loaded master dataset: {len(df):,} rows")

# Define features
feature_cols = [
    'close', 'volume', 'rsi', 'macd', 'macd_signal', 'macd_hist',
    'sma_20', 'sma_50', 'sma_100', 'bb_width', 'bb_percent',
    'atr_percent', 'volume_ratio', 'stoch_k', 'stoch_d', 
    'adx', 'di_plus', 'di_minus', 'momentum', 'price_position',
    'williams_r', 'cci', 'volatility_24h'
]

# Create target (1 if price up in 24h, else 0)
df['target'] = (df.groupby('symbol')['close'].shift(-24) > df['close']).astype(int)
df = df.dropna()

X = df[feature_cols]
y = df['target']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"✅ Training set: {len(X_train):,}")
print(f"✅ Testing set: {len(X_test):,}")

### CELL 3: Train XGBoost

In [ ]:
print("🚀 Training XGBoost model...")

model = XGBClassifier(
    n_estimators=1000,
    max_depth=7,
    learning_rate=0.01,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='gpu_hist',  # Use GPU
    random_state=42
)

model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)

### CELL 4: Evaluate & Save

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

print("\n" + "="*60)
print("📊 XGBOOST RESULTS (TOP 10 CRYPTOS)")
print("="*60)
print(f"Accuracy: {accuracy:.2%}")
print(f"AUC-ROC: {auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Save model
model_path = '/content/drive/MyDrive/crypto_bot/models/xgb_top10_final.pkl'
joblib.dump(model, model_path)

print(f"\n✅ Model saved to: {model_path}")